In [1]:
"""
    Instruções:
        1- Download dos dados municipais do estado de Minas Gerais;
        2- Transformação dos dados de desmatamento;
        3- Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        4- Geração dos arquivos de saída no formato GeoJSON;
"""

'\n    Instruções:\n        1- Download dos dados municipais do estado de Minas Gerais;\n        2- Transformação dos dados de desmatamento;\n        3- Processamento dos dados:\n            -reprojeção para EPSG:31983;\n            -cálculo de área;\n        4- Geração dos arquivos de saída no formato GeoJSON;\n'

# Instalação de Pacotes

In [1]:
# Instalação de Pacotes
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


# Importação de Bibliotecas

In [ ]:
# Importação das bibliotecas
import geopandas as gpd
import pandas as pd
import numpy as np
import html

# Dataset de Municípios de MG

In [36]:
print("🏡 TAREFA 1: Processando municípios de Minas Gerais")
print("=" * 60)

# Carregamento dos dados e conversão de sistema de coordenadas
print("\n📥 Baixando dados dos municípios...")
mun = gpd.read_file("dados/map.geojson").to_crs(epsg=31983)

columns_to_keep = ['id', 'name', 'geometry']
mun = mun[columns_to_keep].rename(columns={'name': "municipio"})

# Cálculo da área em km²
print("\n📐 Calculando área dos municípios...")
mun["area_km2"] = np.round(mun.geometry.area / 10**6, 2)

# Salvando o arquivo
mun.to_file("dados/municipios-mg.geojson", driver='GeoJSON')
print("\n✅ Arquivo salvo em: dados/municipios-mg.geojson")

# Visualização dos dados processados
mun = gpd.read_file("dados/municipios-mg.geojson")
print("\n🔍 Visualizando os dados processados...")
mun

🏡 TAREFA 1: Processando municípios de Minas Gerais

📥 Baixando dados dos municípios...

📐 Calculando área dos municípios...

✅ Arquivo salvo em: dados/municipios-mg.geojson

🔍 Visualizando os dados processados...


,id,municipio,area_km2,geometry
0,3100104,Abadia dos Dourados,895.12,"POLYGON ((241429.802 7990562.876, 242359.697 7..."
1,3100203,Abaeté,1814.50,"POLYGON ((480831.209 7911603.048, 481764.623 7..."
2,3100302,Abre Campo,472.05,"POLYGON ((775002.321 7771717.815, 776244.59 77..."
3,3100401,Acaiaca,100.72,"POLYGON ((696872.185 7747792.052, 697305.64 77..."
4,3100500,Açucena,812.88,"POLYGON ((775332.208 7905556.756, 775804.655 7..."
...,...,...,...,...
848,3171808,Virginópolis,441.42,"POLYGON ((741901.325 7935223.813, 744154.104 7..."
849,3171907,Virgolândia,281.91,"POLYGON ((783014.609 7968328.38, 782934.821 79..."
850,3172004,Visconde do Rio Branco,241.56,"POLYGON ((719981.237 7683148.509, 720339.567 7..."
851,3172103,Volta Grande,208.82,"POLYGON ((755523.495 7598820.567, 755858.99 75..."


# Dados de População e PIB

In [35]:
print("📊 TAREFA 2: População e PIB por Município de Minas Gerais")
print("=" * 60)

# Carregando e tratando o arquivo CSV
print("Carregando dados do CSV...")
dados_ibge = pd.read_csv("dados/dados-mg-mun-ibge.csv", sep=',', skiprows=1)

# Decodificando entidades HTML em todas as colunas de texto
for col in dados_ibge.select_dtypes(include=['object']).columns:
    dados_ibge[col] = dados_ibge[col].astype(str).apply(lambda x: html.unescape(x) if x != 'nan' else x)

# Decodificando nomes das colunas
dados_ibge.columns = [html.unescape(col.strip()) for col in dados_ibge.columns]

print("✅ Entidades HTML decodificadas com sucesso!")

# Filtrando apenas as colunas necessárias: ID, Nome, PIB, População
print("\n🎯 Filtrando colunas necessárias...")
pop_pib = dados_ibge[[
    'Código [-]',
    'Município [-]',
    'PIB per capita - R$ [2021]',
    'População estimada - pessoas [2025]'
]].copy()

# Renomeando colunas para nomes mais limpos
pop_pib.columns = ['id', 'municipio', 'pib', 'populacao']

print("🔧 Convertendo dados para tipos adequados...")

# Convertendo PIB para numérico (tratando vírgulas como separador decimal)
pop_pib['pib'] = pd.to_numeric(pop_pib['pib'].astype(str).str.replace(',', '.'), errors='coerce')

# Convertendo população para numérico
pop_pib['populacao'] = pd.to_numeric(pop_pib['populacao'], errors='coerce')

# Removendo linhas com dados faltantes
linhas_antes = len(pop_pib)
pop_pib = pop_pib.dropna()
linhas_depois = len(pop_pib)

print(f"✅ Dados processados: {linhas_depois} municípios válidos ({linhas_antes - linhas_depois} removidos)")

# Salvando o CSV final
print("\n💾 Salvando arquivo CSV final...")
pop_pib.to_csv("dados/populacao-pib-municipios-mg.csv", index=True)

# Exibindo amostra dos dados finais
print("\n📋 Amostra dos dados finais:")
pop_pib

📊 TAREFA 2: População e PIB por Município de Minas Gerais
Carregando dados do CSV...
✅ Entidades HTML decodificadas com sucesso!

🎯 Filtrando colunas necessárias...
🔧 Convertendo dados para tipos adequados...
✅ Dados processados: 853 municípios válidos (1 removidos)

💾 Salvando arquivo CSV final...

📋 Amostra dos dados finais:


,id,municipio,pib,populacao
0,3100104,Abadia dos Dourados,25174.26,6345.0
1,3100203,Abaeté,23266.10,23173.0
2,3100302,Abre Campo,19553.86,14354.0
3,3100401,Acaiaca,16247.06,3998.0
4,3100500,Açucena,17888.20,8926.0
...,...,...,...,...
848,3171808,Virginópolis,17587.98,10511.0
849,3171907,Virgolândia,11818.51,4471.0
850,3172004,Visconde do Rio Branco,31179.87,40711.0
851,3172103,Volta Grande,26633.91,4441.0


## Juntando o Dataset e Dados de Municípios

In [41]:
print("🗺️ Juntando dados geoespaciais com dados de população e PIB...")
merged = mun.merge(pop_pib, on=['id', 'municipio'], how='left')
merged.to_file("dados/municipios-mg-completo.geojson", driver='GeoJSON')
merged = gpd.read_file("dados/municipios-mg-completo.geojson").to_crs(epsg=31983)

print("\n✅ Arquivo salvo em: dados/municipios-mg-completo.geojson")
merged

🗺️ Juntando dados geoespaciais com dados de população e PIB...

✅ Arquivo salvo em: dados/municipios-mg-completo.geojson


,id,municipio,area_km2,pib,populacao,geometry
0,3100104,Abadia dos Dourados,895.12,25174.26,6345.0,"POLYGON ((241429.802 7990562.876, 242359.697 7..."
1,3100203,Abaeté,1814.50,23266.10,23173.0,"POLYGON ((480831.209 7911603.048, 481764.623 7..."
2,3100302,Abre Campo,472.05,19553.86,14354.0,"POLYGON ((775002.321 7771717.815, 776244.59 77..."
3,3100401,Acaiaca,100.72,16247.06,3998.0,"POLYGON ((696872.185 7747792.052, 697305.64 77..."
4,3100500,Açucena,812.88,17888.20,8926.0,"POLYGON ((775332.208 7905556.756, 775804.655 7..."
...,...,...,...,...,...,...
848,3171808,Virginópolis,441.42,17587.98,10511.0,"POLYGON ((741901.325 7935223.813, 744154.104 7..."
849,3171907,Virgolândia,281.91,11818.51,4471.0,"POLYGON ((783014.609 7968328.38, 782934.821 79..."
850,3172004,Visconde do Rio Branco,241.56,31179.87,40711.0,"POLYGON ((719981.237 7683148.509, 720339.567 7..."
851,3172103,Volta Grande,208.82,26633.91,4441.0,"POLYGON ((755523.495 7598820.567, 755858.99 75..."


# Dados de Desmatamento

In [43]:
print("🌳 TAREFA 3: Processando dados de desmatamento")
print("=" * 60)

# Carregamento dos dados de desmatamento
print("\n📥 Baixando dados de desmatamento...")
df_ago = gpd.read_file("dados/desmatamento_ago22.gpkg")
df_set = gpd.read_file("dados/desmatamento_set22.gpkg")

# Adicionando coluna de mês em ambos os dataframes
print("\n📅 Adicionando coluna de mês...")
df_set['mes'] = 'set'
df_ago['mes'] = 'ago'

# Filtrando colunas relevantes e renomeando
print("\n📋 Filtrando colunas relevantes e renomeando...")
columns_to_keep = ["Bioma", "geometry", "mes"]
df_set = df_set[columns_to_keep].rename(columns={"Bioma": "bioma"})
df_ago = df_ago[columns_to_keep].rename(columns={"Bioma": "bioma"})

# Juntando os dataframes
print("\n🔗 Juntando os dataframes...")
gdf_desmatamento = pd.concat([df_ago, df_set], ignore_index=True)

# Convertendo para EPSG:31983
print("\n🌐 Convertendo para EPSG:31983...")
gdf_desmatamento = gdf_desmatamento.to_crs(epsg=31983)
print("✅ Conversão concluída.")

# Salvando o arquivo final
print("\n💾 Salvando o arquivo final...")
gdf_desmatamento.to_file("dados/focos-desmatamento-mg.geojson", driver="GeoJSON")
print("✅ Arquivo salvo em: dados/focos-desmatamento-mg.geojson")

# Visualização dos dados processados
print("\n🔍 Visualizando os dados processados...")
gdf_desmatamento

🌳 TAREFA 3: Processando dados de desmatamento

📥 Baixando dados de desmatamento...

📅 Adicionando coluna de mês...

📋 Filtrando colunas relevantes e renomeando...

🔗 Juntando os dataframes...

🌐 Convertendo para EPSG:31983...
✅ Conversão concluída.

💾 Salvando o arquivo final...
✅ Arquivo salvo em: dados/focos-desmatamento-mg.geojson

🔍 Visualizando os dados processados...


,bioma,geometry,mes
0,Cerrado,"POLYGON ((65610.256 7779397.013, 65659.326 777...",ago
1,Cerrado,"POLYGON ((31696.271 7842130.438, 31989.678 784...",ago
2,Cerrado,"POLYGON ((50597.259 7823297.562, 50665.831 782...",ago
3,Cerrado,"POLYGON ((50538.446 7823306.388, 50636.036 782...",ago
4,Cerrado,"POLYGON ((67719.523 7822648.62, 67877.128 7822...",ago
...,...,...,...
2034,Mata Atlântica,"POLYGON ((923791.878 8252900.877, 923841.725 8...",set
2035,Mata Atlântica,"POLYGON ((904720.028 8256294.185, 904749.044 8...",set
2036,Mata Atlântica,"POLYGON ((888566.96 8258582.267, 888564.814 82...",set
2037,Mata Atlântica,"POLYGON ((890075.753 8267489.965, 890066.941 8...",set
